We manage 2 bakeries that produce, store, and sell loaves of fresh bread on site. A given
location lcan produce up to loaves of bread every day, and experiences different demand𝑏𝑙𝑑𝑙𝑡
on a daily basis. We are able to store leftover bread to be sold the next day, but only if we sell the
day-old loaves at a discount (with linearly increasing discounts if a loaf is stored for multiple
days), so we effectively incur an opportunity cost of sper loaf that is stored overnight.
Additionally, between one day’s closing and the next day’s opening, we can transport the stored
loaves between the two locations, at a cost of mper loaf.
Our goal is to minimize our costs over a seven day period, while meeting demand at each
location for every day. Assume that we start with 0 loaves of leftover bread on the first day at
both locations.

In [19]:
using Cbc, JuMP, NamedArrays

# Sets
T = [1, 2, 3, 4, 5, 6, 7] # our set of days 
T0 = [0, 1, 2, 3, 4, 5, 6, 7]
L = [1, 2] # bakery locations

# Parameters
c = 0.5 # production cost
s = 0.2 # incurred cost of selling old bread
b = Dict(zip(L, [150, 260])) # maximum # of bread produced in location
# deamnd for bread at location l at day t
d = NamedArray([100 130 150 120 140 220 150; 150 230 200 190 280 140 320], (L, T), ("location", "day")) 
h = 0.1 # transportation cost

########################################
# TODO: create and solve general model #
########################################
using JuMP, Cbc

m = Model()

@variable(m, x[L,T] >= 0) # of bread produced at location L in day T
@variable(m, y[L,T] >= 0) # of leftover bread at location L in day T
@variable(m, z[L,T] >= 0) # of transporated bread from location 1 to 2 in day T

@objective(m, Min, sum(c * x) + sum(s * y) + sum(h * z))

@constraint(m, max_produce[i in L], sum(x[i,j] for j in T) <= b[i]) 
@constraint(m, initial_leftover[i in L], y[i,1] == 0)
#@constraint(m, carryover_1_2, y[1,2] == y[1,1] + x[1,1] - z[1,1])
@constraint(m, carryover_equality[j in T], (y[i,j] for i in L) == (y[i-1,j] + x[i,j] - z[i,j] for i in L))

LoadError: MethodError: no method matching -(::Base.Generator{Vector{Int64}, var"#140#143"{Int64}}, ::Base.Generator{Vector{Int64}, var"#141#144"{Int64}})
[0mClosest candidates are:
[0m  -([91m::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}[39m, ::T) where T at C:\Users\peren\.julia\packages\MathOptInterface\QxT5e\src\Utilities\functions.jl:1764
[0m  -([91m::ChainRulesCore.AbstractThunk[39m, ::Any) at C:\Users\peren\.julia\packages\ChainRulesCore\uxrij\src\tangent_types\thunks.jl:32
[0m  -([91m::ChainRulesCore.ZeroTangent[39m, ::Any) at C:\Users\peren\.julia\packages\ChainRulesCore\uxrij\src\tangent_arithmetic.jl:101
[0m  ...

In [3]:
d

2×7 Named Matrix{Int64}
location ╲ day │   1    2    3    4    5    6    7
───────────────┼──────────────────────────────────
1              │ 100  130  150  120  140  220  150
2              │ 150  230  200  190  280  140  320

In [6]:
x

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7]
And data, a 2×7 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]